In [3]:
#pip install alpaca_trade_api

In [4]:
import pandas as pd
import numpy as np
import json
import time
import websocket
from collections import deque, defaultdict
import alpaca_trade_api as tradeapi
from creds import ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_URL, ALPACA_PAPER
from retrain_model import Model

In [5]:
class AlpacaAPI:
    def __init__(self, api_key, secret_key, base_url):
        self.api = tradeapi.REST(api_key, secret_key, base_url=base_url) # API Initialization
        self.order_queue = deque()  # Queue to track unfilled orders


    def place_order(self, action, qty, symbol):
        """
        Place a market order for a given asset symbol.

        Parameters:
        - action (str): Action to perform, either 'buy' or 'sell'.
        - qty (float): Quantity of the asset to buy or sell.
        - symbol (str): Symbol of the asset (e.g., 'AAPL', 'BTC/USD').

        Returns:
        - order_id (str): The ID of the submitted order.
        """
        try:
            self.order_queue.append(self.api.submit_order(symbol, qty, action, 'market', 'gtc').id)
        except Exception as e:
            print(f"Error submitting order: {e}")


    def get_position_size(self, symbol):
        """
        Get the current position size for a given asset symbol.

        Parameters:
        - symbol (str): Symbol of the asset (e.g., 'AAPL', 'BTC/USD').

        Returns:
        - position_size (float): Current position size. Returns 0 if no position is found or an error occurs.
        """
        try:
            return next((float(position.qty) for position in self.api.list_positions() if position.symbol == symbol), 0)
        except Exception as e:
            print(f"Error getting position size: {e}")
            return 0


    def check_order_status(self):
        """
        Check the status of orders in the order queue.

        Returns:
        - all_orders_filled (bool): True if all orders in the queue are filled, False otherwise.
        """
        while self.order_queue:
            order_id = self.order_queue[0]  # Get the first order in the queue
            order = self.api.get_order(order_id)
            if order.status == 'filled':
                self.order_queue.popleft()  # Remove the filled order from the queue
            else:
                return False  # Order is not filled, exit the function
        return True  # All orders in the queue are filled

In [7]:
class CryptoTradingStrategy:
    def __init__(self, alpaca_api, crypto_pairs, ws_url):
        self.alpaca_api = alpaca_api
        self.parameters = defaultdict(dict)
        self.crypto_pairs = crypto_pairs
        self.ws_url = ws_url
        self.model = Model(self.parameters, self.crypto_pairs)

    def connect_to_websocket(self):
        """
        Connect to the WebSocket and perform authentication and subscription.

        Args:
        - ws_url (str): WebSocket URL.
        - auth_message (dict): Authentication message.
        - subscription_message (dict): Subscription message.

        Returns:
        - ws (WebSocket): Connected WebSocket object.
        """
        ws = websocket.create_connection(self.ws_url)
        auth_message = {"action": "auth", "key": ALPACA_API_KEY, "secret": ALPACA_SECRET_KEY}
        subscription_message = {"action": "subscribe", "quotes": self.crypto_pairs}
        ws.send(json.dumps(auth_message))
        ws.send(json.dumps(subscription_message))
        return ws

    def calculate_and_print_moving_averages(self, pair, bid_price_data, ma_fast, ma_slow, parameters):
        """
        Calculate and print moving averages for a given trading pair.

        Parameters:
        - pair (str): The trading pair symbol (e.g., 'BTC/USD').
        - bid_price_data (dict): Dictionary containing bid price data for different trading pairs.
        - ma_fast (dict): Dictionary containing moving averages (fast) for different trading pairs.
        - ma_slow (dict): Dictionary containing moving averages (slow) for different trading pairs.
        - parameters (dict): Dictionary containing trading parameters for different trading pairs.
        """
        ma_fast[pair] = np.mean(bid_price_data[pair][-parameters[pair]['ma_fast']:])
        ma_slow[pair] = np.mean(bid_price_data[pair][-parameters[pair]['ma_slow']:])
        print(f"ma_fast: {ma_fast[pair]}\nma_slow: {ma_slow[pair]}")

    def evaluate_trading_signals(self, pair, ma_fast, ma_slow, ma_fast_prev, ma_slow_prev):

        """
        Evaluate trading signals based on moving averages and execute buy or sell orders.

        Parameters:
        - pair (str): The trading pair symbol (e.g., 'BTC/USD').
        - ma_fast (dict): Dictionary containing moving averages (fast) for different trading pairs.
        - ma_slow (dict): Dictionary containing moving averages (slow) for different trading pairs.
        - ma_fast_prev (dict): Dictionary containing previous moving averages (fast) for different trading pairs.
        - ma_slow_prev (dict): Dictionary containing previous moving averages (slow) for different trading pairs.
        """

        if self.model.predict_probability(pair, [ma_slow, ma_fast, int(ma_fast > ma_slow)],1) > 0.5:
            self.execute_buy_order(pair)
        elif self.model.predict_probability(pair, [ma_slow, ma_fast, int(ma_fast > ma_slow)],-1) > 0:
            self.execute_sell_order(pair)
        else:
            print("Holding position")

    def execute_buy_order(self, pair):
        """
        Execute a buy order for a given trading pair if no position is currently held.

        Parameters:
        - pair (str): The trading pair symbol (e.g., 'BTC/USD').
        """
        if self.alpaca_api.get_position_size(pair) == 0:
            print("buy order")
            self.alpaca_api.place_order('buy', 0.01, pair)

    def execute_sell_order(self, pair):
        """
        Execute a sell order for a given trading pair if a position is currently held.

        Parameters:
        - pair (str): The trading pair symbol (e.g., 'BTC/USD').
        """
        position_size = self.alpaca_api.get_position_size(pair)
        if position_size > 0:
            print("sell order")
            self.alpaca_api.place_order('sell', position_size, pair)

    def retrain(self, pair, temp_data, model, parameters):
        """
        Retrain the trading model for a given pair using temporary data.

        Parameters:
        - pair (str): The trading pair symbol (e.g., 'BTC/USD').
        - temp_data (dict): Dictionary containing temporary data for different trading pairs.
        - model (Model): The trading model instance.
        - parameters (dict): Dictionary containing trading parameters for different trading pairs.
        """
        if len(temp_data[pair]) >= 13:  # parameter to be defined:
            temp_df = pd.DataFrame(temp_data[pair])
            model.retrain_strat(pair, temp_df)
            print(f"Retrained the model for {pair}")

            model.delete_old_records(pair, num_to_keep=800)

    def run_strategy(self):
        """
        Run the trading strategy.

        Connects to the Alpaca WebSocket, receives real-time price data for specified crypto pairs,
        and executes the trading strategy based on moving averages. It continuously updates temporary
        and historical data, evaluates trading signals, and re-trains the model.

        Raises:
        - Exception: If an error occurs during the execution of the strategy.

        Notes:
        - The strategy runs indefinitely until an exception is encountered.
        - The strategy uses moving averages to generate buy and sell signals.
        """
        ws = self.connect_to_websocket()

        temp_data = {pair: [] for pair in self.crypto_pairs}
        bid_price_data = defaultdict(list)
        ma_fast = defaultdict(float)
        ma_slow = defaultdict(float)
        ma_fast_prev = defaultdict(float)
        ma_slow_prev = defaultdict(float)

        ## These are the ideal parameters from backtesting
        ## We will start with them and retrain the model
        # Ideal parameters from backtesting

        self.parameters = {
            'BTC/USD': {'max_depth': 25, 'min_samples_leaf': 300},
            'ETH/USD': {'max_depth': 25, 'min_samples_leaf': 1000},
            'LTC/USD': {'max_depth': 5, 'min_samples_leaf': 500}
        }

        try:
            while True:
                response = ws.recv()
                data_dict = json.loads(response)[0]
                print("data", data_dict)
                if 'S' in data_dict and data_dict['S'] in self.crypto_pairs and 'bp' in data_dict and 't' in data_dict:
                    print(f"Received price for {data_dict['S'], data_dict['bp']}")
                    pair = data_dict['S']
                    ## We will also need to extract more data such as volume, etc.
                    ## This way we can check for data skewness
                    temp_data[pair].append({'Time': data_dict['t'], 'close': data_dict['bp']})
                    bid_price_data[pair].append(data_dict['bp'])

                    if ma_fast[pair] and ma_slow[pair]:
                        ma_fast_prev[pair], ma_slow_prev[pair] = ma_fast[pair], ma_slow[pair]
                        print(f"ma_fast_prev: {ma_fast_prev[pair]} \n ma_slow_prev: {ma_slow_prev[pair]}")


                    self.evaluate_trading_signals(pair, ma_fast, ma_slow, ma_fast_prev, ma_slow_prev)
                    self.retrain(pair, temp_data, self.model, self.parameters)

                    ## Model is trained on minute data, so we should receive at more or less minute freq.
                    time.sleep(0.1)


        except Exception as e:
            print(f"Exception in strategy: {e}")

        finally:
            if ws:
                ws.close()


data {'T': 'success', 'msg': 'connected'}
data {'T': 'success', 'msg': 'authenticated'}
data {'T': 'subscription', 'trades': [], 'quotes': ['BTC/USD', 'ETH/USD', 'LTC/USD'], 'orderbooks': [], 'bars': [], 'updatedBars': [], 'dailyBars': []}
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.67, 'bs': 98.47, 'ap': 73.07, 'as': 48.863, 't': '2023-12-14T00:23:42.359575566Z'}
Received price for ('LTC/USD', 72.67)
Exception in strategy: '>' not supported between instances of 'collections.defaultdict' and 'collections.defaultdict'
data {'T': 'success', 'msg': 'connected'}
data {'T': 'success', 'msg': 'authenticated'}
data {'T': 'subscription', 'trades': [], 'quotes': ['BTC/USD', 'ETH/USD', 'LTC/USD'], 'orderbooks': [], 'bars': [], 'updatedBars': [], 'dailyBars': []}
data {'T': 'q', 'S': 'BTC/USD', 'bp': 42756.31, 'bs': 0.55468, 'ap': 42812.461, 'as': 0.2758, 't': '2023-12-14T00:23:44.876919646Z'}
Received price for ('BTC/USD', 42756.31)
Exception in strategy: '>' not supported between instances of 'col

KeyboardInterrupt: ignored

In [ ]:
if __name__ == "__main__":
    alpaca_api = AlpacaAPI(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_PAPER)
    crypto_pairs = ['BTC/USD', 'ETH/USD', 'LTC/USD']
    ws_url = ALPACA_URL

    crypto_strategy = CryptoTradingStrategy(alpaca_api, crypto_pairs, ws_url)

    failed_times = 0
    while True:
        try:
            crypto_strategy.run_strategy()
        except Exception as e:
            print(f"Exception in main loop: {e}")
            failed_times += 1
            if failed_times > 100:
                print("Too many failures, stopping.")
                break
            time.sleep(5)
            print(f"Failed times: {failed_times}")

data {'T': 'success', 'msg': 'connected'}
data {'T': 'success', 'msg': 'authenticated'}
data {'T': 'subscription', 'trades': [], 'quotes': ['BTC/USD', 'ETH/USD', 'LTC/USD'], 'orderbooks': [], 'bars': [], 'updatedBars': [], 'dailyBars': []}
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.348, 'bs': 97.82, 'ap': 72.73, 'as': 49.26, 't': '2023-12-12T23:27:20.551223759Z'}
Received price for ('LTC/USD', 72.348)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.492, 'bs': 48.718, 'ap': 72.73, 'as': 49.26, 't': '2023-12-12T23:27:20.551386919Z'}
Received price for ('LTC/USD', 72.492)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.492, 'bs': 48.718, 'ap': 72.914, 'as': 97.4, 't': '2023-12-12T23:27:20.551421999Z'}
Received price for ('LTC/USD', 72.492)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.492, 'bs': 48.718, 'ap': 72.71, 'as': 48.7798, 't': '2023-12-12T23:27:20.551626149Z'}
Received price for ('LTC/USD', 72.492)
Holding position
data {'T': 'q', 'S': 'BTC/USD', 'bp

Time taken to retrain model for LTC/USD: 5.055432319641113 seconds
model data 5
parameters after retraining for LTC/USD: {'ma_slow': 100, 'ma_fast': 50}
Retrained the model for LTC/USD
number of records for the pair 5
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.51, 'bs': 48.46, 'ap': 72.71, 'as': 48.7798, 't': '2023-12-12T23:27:27.015848894Z'}
Received price for ('LTC/USD', 72.51)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.51, 'bs': 48.46, 'ap': 72.91, 'as': 96.7099, 't': '2023-12-12T23:27:27.016169294Z'}
Received price for ('LTC/USD', 72.51)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.51, 'bs': 48.46, 'ap': 72.682, 'as': 48.7673, 't': '2023-12-12T23:27:27.016220484Z'}
Received price for ('LTC/USD', 72.51)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.29, 'bs': 97.3004, 'ap': 72.682, 'as': 48.7673, 't': '2023-12-12T23:27:33.243686101Z'}
Received price for ('LTC/USD', 72.29)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.54, 'bs': 48.8101

Time taken to retrain model for LTC/USD: 4.290108919143677 seconds
model data 10
parameters after retraining for LTC/USD: {'ma_slow': 100, 'ma_fast': 50}
Retrained the model for LTC/USD
number of records for the pair 10
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.54, 'bs': 48.8101, 'ap': 72.88, 'as': 98.2553, 't': '2023-12-12T23:27:33.243750121Z'}
Received price for ('LTC/USD', 72.54)
Holding position
data {'T': 'q', 'S': 'LTC/USD', 'bp': 72.54, 'bs': 48.8101, 'ap': 72.696, 'as': 48.419, 't': '2023-12-12T23:27:33.243932371Z'}
Received price for ('LTC/USD', 72.54)
Holding position


KeyboardInterrupt: ignored